<a href="https://colab.research.google.com/github/aehyok/go-openai/blob/main/%E2%80%9Cwhisper_m4a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

直接将m4a音频文件转换为字幕文件srt





In [ ]:
#@markdown # **检查GPU硬件加速** 🕵️

#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@markdown ---

!nvidia-smi -L

!nvidia-smi
     

GPU 0: Tesla T4 (UUID: GPU-30297e83-80fd-9354-3339-12897273eb98)
Mon Mar 27 11:44:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

In [ ]:
#@markdown **配置Whisper/Setup Whisper** 🏗️

!pip install requests beautifulsoup4
!pip install git+https://github.com/openai/whisper.git

import torch
import sys

device = torch.device('cuda:0')
print('正在使用的设备:', device, file=sys.stderr)

print('Whisper已经被安装请执行下一个单元')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6p9at6gc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-6p9at6gc
  Resolved https://github.com/openai/whisper.git to commit 6dea21fd7f7253bfe450f1e2512a0fe47ee2d258
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Whisper已经被安装请执行下一个单元


正在使用的设备: cuda:0


In [ ]:
# @markdown # ** whisper Model选择 ** 🧠


Model = 'large-v2' #@param ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large', 'large-v2']
#@markdown ---

import whisper
from IPython.display import Markdown

whisper_model = whisper.load_model(Model)

if Model in whisper.available_models():
    display(Markdown(
        f"**{Model} model已经被选择了**"
    ))
else:
    display(Markdown(
        f"**{Model} model is no longer available.** Please select one of the following: - {' - '.join(whisper.available_models())}"
    ))

**large-v2 model已经被选择了**

In [ ]:
#@markdown # **开始运行model** 🚀

#@markdown Run this cell to execute the transcription of the video. This can take a while and very based on the length of the video and the number of parameters of the model selected above.

#@markdown ## **Parameters** ⚙️

#@markdown ### **Behavior control**
#@markdown ---
language = "zh" ## 我这里直接写死为中文的
#@markdown ---
verbose = 'Live transcription' #@param ['Live transcription', 'Progress bar', 'None']
#@markdown > Whether to print out the progress and debug messages.
#@markdown ---
output_format = 'srt' #@param ['txt', 'vtt', 'srt', 'tsv', 'json', 'all']
#@markdown > 要生成哪种类型的字幕文件？

#@markdown ---
task = 'transcribe' #@param ['transcribe', 'translate']
#@markdown > Whether to perform X->X speech recognition (`transcribe`) or X->English translation (`translate`).
#@markdown ---

#@markdown 

#@markdown ### **Optional: Fine tunning** 
#@markdown ---
temperature = 0.2 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown > Temperature to use for sampling.
#@markdown ---
temperature_increment_on_fallback = 0.2 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown > Temperature to increase when falling back when the decoding fails to meet either of the thresholds below.
#@markdown ---
best_of = 5 #@param {type:"integer"}
#@markdown > Number of candidates when sampling with non-zero temperature.
#@markdown ---
beam_size = 8 #@param {type:"integer"}
#@markdown > Number of beams in beam search, only applicable when temperature is zero.
#@markdown ---
patience = 1.0 #@param {type:"number"}
#@markdown > Optional patience value to use in beam decoding, as in [*Beam Decoding with Controlled Patience*](https://arxiv.org/abs/2204.05424), the default (1.0) is equivalent to conventional beam search.
#@markdown ---
length_penalty = -0.05 #@param {type:"slider", min:-0.05, max:1, step:0.05}
#@markdown > Optional token length penalty coefficient (alpha) as in [*Google's Neural Machine Translation System*](https://arxiv.org/abs/1609.08144), set to negative value to uses simple length normalization.
#@markdown ---
suppress_tokens = "-1" #@param {type:"string"}
#@markdown > Comma-separated list of token ids to suppress during sampling; '-1' will suppress most special characters except common punctuations.
#@markdown ---
initial_prompt = "" #@param {type:"string"}
#@markdown > Optional text to provide as a prompt for the first window.
#@markdown ---
condition_on_previous_text = True #@param {type:"boolean"}
#@markdown > if True, provide the previous output of the model as a prompt for the next window; disabling may make the text inconsistent across windows, but the model becomes less prone to getting stuck in a failure loop.
#@markdown ---
fp16 = True #@param {type:"boolean"}
#@markdown > whether to perform inference in fp16.
#@markdown ---
compression_ratio_threshold = 2.4 #@param {type:"number"}
#@markdown > If the gzip compression ratio is higher than this value, treat the decoding as failed.
#@markdown ---
logprob_threshold = -1.0 #@param {type:"number"}
#@markdown > If the average log probability is lower than this value, treat the decoding as failed.
#@markdown ---
no_speech_threshold = 0.6 #@param {type:"slider", min:-0.0, max:1, step:0.05}
#@markdown > If the probability of the <|nospeech|> token is higher than this value AND the decoding has failed due to `logprob_threshold`, consider the segment as silence.
#@markdown ---

verbose_lut = {
    'Live transcription': True,
    'Progress bar': False,
    'None': None
}

import numpy as np
import warnings
import shutil
from pathlib import Path

args = dict(
    language = (None if language == "Auto detection" else language),
    verbose = verbose_lut[verbose],
    task = task,
    temperature = temperature,
    temperature_increment_on_fallback = temperature_increment_on_fallback,
    best_of = best_of,
    beam_size = beam_size,
    patience=patience,
    length_penalty=(length_penalty if length_penalty>=0.0 else None),
    suppress_tokens=suppress_tokens,
    initial_prompt=(None if not initial_prompt else initial_prompt),
    condition_on_previous_text=condition_on_previous_text,
    fp16=fp16,
    compression_ratio_threshold=compression_ratio_threshold,
    logprob_threshold=logprob_threshold,
    no_speech_threshold=no_speech_threshold
)

temperature = args.pop("temperature")
temperature_increment_on_fallback = args.pop("temperature_increment_on_fallback")
if temperature_increment_on_fallback is not None:
    temperature = tuple(np.arange(temperature, 1.0 + 1e-6, temperature_increment_on_fallback))
else:
    temperature = [temperature]

if Model.endswith(".en") and args["language"] not in {"en", "English"}:
    warnings.warn(f"{Model} is an English-only model but receipted '{args['language']}'; using English instead.")
    args["language"] = "en"
title ="test1"
audio_path = "/content/downloads/test1.m4a"
audio_path_local = Path(audio_path).resolve()
print("audio local path:", audio_path_local)
print("audio audio_path:", audio_path)
transcription = whisper.transcribe(
    whisper_model,
    str(audio_path_local),
    temperature=temperature,
    **args,
)

# Save output
whisper.utils.get_writer(
    output_format=output_format,
    output_dir=audio_path_local.parent
)(
    transcription,
    title
)

try:
    if output_format=="all":
        for ext in ('txt', 'vtt', 'srt', 'tsv', 'json'):
            transcript_file_name = title + "." + ext
            display(Markdown(f"**Transcript file created: {transcript_file_name}**"))
    else:
        transcript_file_name = title + "." + output_format

        display(Markdown(f"**Transcript file created: {transcript_file_name}**"))

except:
    display(Markdown(f"**Transcript file created: {transcript_local_path}**"))



audio local path: /content/downloads/test1.m4a
audio audio_path: /content/downloads/test1.m4a
[00:00.000 --> 00:04.100] 你好,我是咱们专栏的学习委员朱英达
[00:04.100 --> 00:06.500] 在预备知识片这个模块呢
[00:06.500 --> 00:08.700] 龚老师给我们系统地梳理了
[00:08.700 --> 00:11.500] 编译过程中各个阶段的核心要点
[00:11.500 --> 00:12.800] 主要的目的呢
[00:12.800 --> 00:16.500] 就是让我们建立一个编译原理的知识体系
[00:16.500 --> 00:18.300] 那么到今天为止呢
[00:18.300 --> 00:20.500] 我们就学完这部分内容了
[00:20.500 --> 00:23.700] 那你对这些知识掌握得如何呢
[00:23.700 --> 00:27.300] 为了复习,也为了检测我们的学习成果
[00:27.500 --> 00:30.600] 我就根据自己的积累和一些学习经验
[00:30.600 --> 00:32.800] 整理了一张知识地图
[00:32.800 --> 00:37.100] 你呢,可以根据这张地图中标记的七大编译阶段
[00:37.100 --> 00:38.800] 用来随时速查那些
[00:38.800 --> 00:42.700] 我们经常使用的编译原理概念和关键算法
[00:42.700 --> 00:45.700] 如果你也总结了自己的知识地图呢
[00:45.700 --> 00:48.200] 那你也可以对照着我整理的这个
[00:48.200 --> 00:49.700] 给自己一个反馈
[00:49.700 --> 00:52.000] 看看它们之间有哪些一样
[00:52.000 --> 00:53.800] 或者是不一样的地方
[00:53.800 --> 00:56.700] 我们可以在留言区一起进行交流
[00:56.700 --> 00:59.300] 不过啊,虽然知识地图的形式
[00:59.300 --> 01:02.100] 比较方便

**Transcript file created: test1.srt**